In [1]:
import sys
import pandas as pd
import os
import re
import glob
import shutil
import numpy as np
#import matplotlib.pyplot as plt
#import skimage
from pathlib import Path
import shutil
from base64 import b64encode
from base64 import b64decode

#plt.rcParams["figure.figsize"] = (10,10)

from IPython.core.display import display, HTML
# make the Jupyter notebook use the full screen width
display(HTML("<style>.container { width:99% !important; }</style>"))

In [2]:
PATH="/notebook/nas-trainings/arne/DGFISMA/DATA/"

In [3]:
import random
random.seed(10)


definitions=open( os.path.join( PATH , "definition_extraction/bootstrapped_training_set/iteration_1/training_set_def" ) ).read().strip("\n").split("\n")
no_definitions=open( os.path.join( PATH , "definition_extraction/bootstrapped_training_set/iteration_1/training_set_no_def" ) ).read().strip("\n").split("\n")

training_set=[]

for definition in definitions:
    training_set.append( (definition, 1 ) )

for no_definition in no_definitions:
    training_set.append( (  no_definition, 0 ) )
        
        
wikipedia_def=open( os.path.join( PATH, "definition_extraction/bootstrapped_training_set/wikipedia_definitions" )    ).read().strip("\n").split("\n")
for sentence in wikipedia_def:
    if "/DEF" in sentence:
        sentence=sentence.replace( "/TERM", "").replace( "/DEF", "" ).replace( "/O", "" ).replace( "-LBR-" , "(" ).replace( "-RBR-", ")" ).replace( "&amp", "&").replace( "&nbsp", " ")
        training_set.append( ( sentence, 1  )   )
    else:
        training_set.append( ( sentence, 0  )   )
        
        
random.shuffle( training_set )

valid_set=training_set[ :200  ]
train_set=training_set[  200: ]

with open( os.path.join(  PATH, "definition_extraction/bootstrapped_training_set/iteration_1/train_fasttext.txt" ) , "w" ) as f:
    for text, label in train_set:
        if label==1:
            f.write( f'__label__definition {text}\n'   )
        else:
            f.write( f'__label__no_definition {text}\n'   )
            
with open( os.path.join(  PATH, "definition_extraction/bootstrapped_training_set/iteration_1/valid_fasttext.txt" )   , "w" ) as f:
    for text, label in valid_set:
        if label==1:
            f.write( f'__label__definition {text}\n'   )
        else:
            f.write( f'__label__no_definition {text}\n'   )

In [5]:
import fasttext
model = fasttext.train_supervised( input= os.path.join(  PATH, "definition_extraction/bootstrapped_training_set/iteration_1/train_fasttext.txt" ), 
                                 epoch=200, 
                                dim=100,
                                  minCount=5,
                                  ws=1,
                                 )
model.save_model( os.path.join( PATH, "definition_extraction/bootstrapped_training_set/iteration_1/model_fasttext.bin" ) )

In [6]:
from sklearn.metrics import classification_report

model = fasttext.load_model( os.path.join( PATH, "definition_extraction/bootstrapped_training_set/iteration_1/model_fasttext.bin" )  )

#sanity check:
assert( set(model.labels) == set( [ "__label__no_definition" , "__label__definition" ] )  )

test_sentences=open(  os.path.join( PATH, "definition_extraction/bootstrapped_training_set/iteration_1/test_sentences" )    ).read().strip( "\n" ).split("\n")
pred_labels=model.predict( test_sentences  )[0]
pred_labels=[ 1 if label[0] == "__label__definition" else 0 for label in pred_labels  ]  #explicitely use the labels 

test_labels=open( os.path.join( PATH, "definition_extraction/bootstrapped_training_set/iteration_1/test_labels" )  ).read().strip( "\n" ).split( "\n" )
test_labels=[ int(label)  for label in test_labels  ]

print( classification_report( test_labels, pred_labels  )  )

              precision    recall  f1-score   support

           0       0.73      0.83      0.77        29
           1       0.80      0.69      0.74        29

   micro avg       0.76      0.76      0.76        58
   macro avg       0.76      0.76      0.76        58
weighted avg       0.76      0.76      0.76        58



In [ ]:
      precision    recall  f1-score   support

           0       0.73      0.83      0.77        29
           1       0.80      0.69      0.74        29

   micro avg       0.76      0.76      0.76        58
   macro avg       0.76      0.76      0.76        58
weighted avg       0.76      0.76      0.76        58


In [26]:
try:
    assert( set(model.labels) == set( [ "__label__no_definitions" , "__label__definition" ] )  )
except AssertionError:
    print("df")
    raise ValueError(f'Fasttext model labels {model.labels} should match "pos_label" and "neg_label".'  )

df


ValueError: Fasttext model labels ['__label__no_definition', '__label__definition'] should match "pos_label" and "neg_label".